# Data Preparation

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install jiwer 

In [ ]:
from google.cloud import storage
import os
import glob
import zipfile
import pandas as pd
import numpy as np
import librosa

In [ ]:
!gsutil cp gs://recitation_dataset/data.zip .

In [ ]:
!unzip data.zip -d .

In [ ]:
!rm -rf "data/bismillah/.ipynb_checkpoints"

In [ ]:
if not os.path.exists('saved_model'):
    os.mkdir('saved_model')

In [ ]:
df = pd.read_csv('Arabic-Original.csv', delimiter='|', header=None)
df.columns = ['surah_number', 'aayah_number', 'aayah']
#df.drop(['surah_number', 'aayah_number'], axis=1, inplace=True)
df.head(10)

In [ ]:
dataset = None

for qari in os.listdir('data/bismillah'):
    audio_files = glob.glob(f'data/bismillah/{qari}/*.wav')
    df_2 = pd.DataFrame({'path' : audio_files})

    df_path = (
        df['path'].str.rsplit('/', n=1, expand=True)[1]
        .str.replace('.wav', '')
        .str.split('_', expand=True)
        .rename(columns={0: 'surah_number', 1: 'aayah_number'})
        .astype({"surah_number":"int","aayah_number":"int"})
    )
    df_path['path'] = df_2['path'] 
    
    df_out = pd.merge(df, df_path, on=["surah_number", "aayah_number"]).drop(columns=['surah_number', 'aayah_number'])

    if dataset is None:
        dataset = df_out
    else:
        dataset = pd.concat([dataset, df_out]).reset_index(drop=True)    
        
dataset = dataset.rename(columns={'aayah': 'transcription'})

In [ ]:
len(dataset)

In [ ]:
dataset.to_csv('final_dataset_linux.csv', index=False)

# Training

In [ ]:
from transformers import AutoProcessor, AutoModelForCTC
from datasets import Dataset, load_dataset, Audio, DatasetDict
import pandas as pd
import json
import os

In [ ]:
from transformers import AutoFeatureExtractor
from transformers import AutoConfig

processor = AutoProcessor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-arabic")
model = AutoModelForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-arabic")

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
dataset = load_dataset('csv', data_files='final_dataset_linux.csv', split='train')
dataset = dataset.rename_columns({"path": "audio", "transcription": "text"})
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))
dataset = dataset.map(prepare_dataset).remove_columns(["audio", "text"])

In [ ]:
dataset

In [ ]:
train_test_dataset = dataset.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_test_dataset['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [ ]:
if not os.path.exists("processed_dataset"):
    os.mkdir("processed_dataset")

train_test_valid_dataset.save_to_disk("processed_dataset/")

In [ ]:
from datasets import load_from_disk

dataset = load_from_disk("processed_dataset/")

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union



@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='saved_model',
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=10,
  push_to_hub=False,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
if not os.path.exists("final_product"):
    os.mkdir("final_product")
trainer.save_model('final_product/')

In [ ]:
dataset['test']